In [2]:
import os

BASE_DIR = "/content/drive/MyDrive/Colab/gtsrb"
MODEL_PATH = os.path.join(BASE_DIR, "traffic_sign_classifier.pth")
DATA_DIR = os.path.join(BASE_DIR, "data")

# Ensure directories exist
os.makedirs(DATA_DIR, exist_ok=True)

In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
train_dataset = datasets.GTSRB(root=DATA_DIR, split='train', download=True, transform=transform)
test_dataset = datasets.GTSRB(root=DATA_DIR, split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training Set: {len(train_dataset)} images")
print(f"Test Set: {len(test_dataset)} images")

100%|██████████| 187M/187M [00:12<00:00, 15.0MB/s]


Extracting /content/drive/MyDrive/Colab/gtsrb/data/gtsrb/GTSRB-Training_fixed.zip to /content/drive/MyDrive/Colab/gtsrb/data/gtsrb


100%|██████████| 89.0M/89.0M [00:08<00:00, 10.0MB/s]


Extracting /content/drive/MyDrive/Colab/gtsrb/data/gtsrb/GTSRB_Final_Test_Images.zip to /content/drive/MyDrive/Colab/gtsrb/data/gtsrb


100%|██████████| 99.6k/99.6k [00:00<00:00, 151kB/s]

Extracting /content/drive/MyDrive/Colab/gtsrb/data/gtsrb/GTSRB_Final_Test_GT.zip to /content/drive/MyDrive/Colab/gtsrb/data/gtsrb
Training Set: 26640 images
Test Set: 12630 images


In [4]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TrafficSignCNN(nn.Module):
    def __init__(self, num_classes=43):  # GTSRB has 43 classes
        super(TrafficSignCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = TrafficSignCNN().to(device)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

    # Save model checkpoint after every epoch
    torch.save(model.state_dict(), MODEL_PATH)
    print(f"Model saved at {MODEL_PATH}")

Epoch 1/10, Loss: 1.4687, Accuracy: 58.33%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 2/10, Loss: 0.2394, Accuracy: 93.35%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 3/10, Loss: 0.1191, Accuracy: 96.84%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 4/10, Loss: 0.0642, Accuracy: 98.37%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 5/10, Loss: 0.0443, Accuracy: 98.87%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 6/10, Loss: 0.0383, Accuracy: 99.04%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 7/10, Loss: 0.0328, Accuracy: 99.12%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 8/10, Loss: 0.0179, Accuracy: 99.61%
Model saved at /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
Epoch 9/10, Loss: 0.0220, Accura

In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 89.51%


In [9]:
torch.save(model.state_dict(), MODEL_PATH)
print(f"Final model saved at: {MODEL_PATH}")

Final model saved at: /content/drive/MyDrive/Colab/gtsrb/traffic_sign_classifier.pth
